<img width="100" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Forest Emissions Tracking - Validation

_CarbonPlan ClimateTrace Team_

This notebook compares our estimates of country-level forest emissions to prior estimates from other
groups. The notebook currently compares againsts:

- Global Forest Watch (Zarin et al. 2016)
- Global Carbon Project (Friedlingstein et al. 2020)


In [ ]:
import geopandas
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import xarray as xr
from carbonplan_styles.mpl import set_theme
from carbonplan_styles.colors import colors
from carbonplan_trace.v1.emissions_workflow import open_fire_mask
import urllib3
import numpy as np

urllib3.disable_warnings()
set_theme()

# load in the 3km rasters of v1 and v0


In [ ]:
coarse_v1 = xr.open_zarr("s3://carbonplan-climatetrace/v1.2/results/global/3000m/raster_split.zarr")
coarse_v0 = xr.open_zarr("s3://carbonplan-climatetrace/v0.4/global/3000m/raster_split.zarr")

In [ ]:
coarse_v0 = coarse_v0.assign_coords({"lat": coarse_v1.lat.values, "lon": coarse_v1.lon.values})

# load in a sample 30m tile (this one covers the PNW)


In [ ]:
tile = "50N_130W"
version = "v1.2"
biomass_30m = xr.open_zarr(f"s3://carbonplan-climatetrace/{version}/results/tiles/{tile}.zarr")
biomass_30m = biomass_30m.rename({"time": "year"})
biomass_30m = biomass_30m.assign_coords({"year": np.arange(2014, 2021)})
emissions_30m_v0 = xr.open_zarr(f"s3://carbonplan-climatetrace/v0.4/tiles/30m/{tile}_tot.zarr/")
emissions_30m_v0 = emissions_30m_v0.rename({"emissions": "Emissions [tCO2/ha]"})
emissions_30m_v1 = xr.open_zarr(
    f"s3://carbonplan-climatetrace/{version}/results/tiles/30m/{tile}_split.zarr/"
)
emissions_30m_v1["Emissions\n[tCO2/ha]"] = (
    emissions_30m_v1["emissions_from_clearing"] + emissions_30m_v1["emissions_from_fire"]
)
min_lat, max_lat, min_lon, max_lon = 47.55, 47.558, -121.834, -121.82  # north bend
# min_lat, max_lat, min_lon, max_lon = -6.32,-6.318, -53.446,-53.445#amazon

small_subset = {"lat": slice(min_lat, max_lat), "lon": slice(min_lon, max_lon)}
small_subset_reversed = {"lat": slice(max_lat, min_lat), "lon": slice(min_lon, max_lon)}

In [ ]:
def format_panel_plot(fg, titles, label_coords):
    for year, panel in zip(titles, fg.axes[0]):
        panel.plot(
            label_coords["lon"],
            label_coords["lat"],
            marker="o",
            fillstyle="none",
            markeredgecolor="blue",
        )
        panel.set_xlabel("")
        panel.set_ylabel("")
        panel.set_title(year)
        panel.ticklabel_format(useOffset=False)

# make the figures that go into the v1 methods doc where we track how biomass changes and how that translates into emissions


In [ ]:
sample_pixel = {"lat": 47.554, "lon": -121.825}

In [ ]:
fg = (
    biomass_30m.sel(small_subset)
    .rename({"AGB": "AGB [t/ha]"})["AGB [t/ha]"]
    .plot(col="year", vmax=200, vmin=0, cmap="earth_light", figsize=(22, 3))
)
format_panel_plot(fg, biomass_30m.year.values, sample_pixel)

In [ ]:
fg = (
    emissions_30m_v1.sel(small_subset)
    .rename({"Emissions\n[tCO2/ha]": "Emissions - v1 - [tCO2/ha]"})["Emissions - v1 - [tCO2/ha]"]
    .plot(col="year", vmax=400, cmap="fire_light", figsize=(19, 3))
)
format_panel_plot(fg, emissions_30m_v1.year.values, sample_pixel)

In [ ]:
fg = (
    emissions_30m_v1.sel(small_subset)
    .rename({"sinks": "Sinks [tCO2/ha]"})["Sinks [tCO2/ha]"]
    .plot(col="year", cmap="water_light_r", figsize=(19, 3))
)
format_panel_plot(fg, emissions_30m_v1.year.values, sample_pixel)

# compare those emissions to the same region in v0 (emissions only come off in 2018 since that is the only year with a loss according to hansen)


In [ ]:
fg = (
    emissions_30m_v0.sel(small_subset_reversed)
    .sel(year=slice(2015, 2020))
    .rename({"Emissions [tCO2/ha]": "Emissions - v0 - [tCO2/ha]"})["Emissions - v0 - [tCO2/ha]"]
    .plot(col="year", vmax=400, cmap="fire_light", figsize=(19, 3))
)
format_panel_plot(fg, emissions_30m_v0.sel(year=slice(2015, 2020)).year.values, sample_pixel)

# look at an individual pixel and track how the carbon pools change over time


In [ ]:
fig, axarr = plt.subplots(nrows=2, sharex=True)
all_carbon_pools.sel(north_bend_cell).plot(label="Total carbon pool", ax=axarr[0], color="k")
biomass_30m.sel(north_bend_cell).AGB.plot(label="AGB", ax=axarr[0])
biomass_30m.sel(north_bend_cell).BGB.plot(label="BGB", ax=axarr[0])
biomass_30m.sel(north_bend_cell).dead_wood.plot(label="Dead wood", ax=axarr[0])
biomass_30m.sel(north_bend_cell).litter.plot(label="Litter", ax=axarr[0])
all_carbon_pools = (
    biomass_30m["AGB"] + biomass_30m["BGB"] + biomass_30m["dead_wood"] + biomass_30m["litter"]
)
ax = axarr[0].set_ylabel("Biomass\n[t/hectare]")
emissions_30m_v1["Emissions\n[tCO2/ha]"].sel(north_bend_cell).plot(
    label="Emissions", ax=axarr[1], color="k", linestyle="--"
)
axarr[1].set_ylabel("Emissions\n[tCO2/ha]")
lines, labels = [], []
for ax in axarr:
    axLine, axLabel = ax.get_legend_handles_labels()
    ax.set_title("")
    ax.set_xlabel("")
    lines.extend(axLine)
    labels.extend(axLabel)
# fig.legend(lines, labels,
#            loc = 'upper right')
fig.legend(lines, labels, bbox_to_anchor=(0, 0), loc="upper left", ncol=3)

plt.xlabel("")
plt.savefig("single_cell_timeseries.png", dpi=200)

# then compare the emissions for that pixel between v0 and v1


In [ ]:
emissions_30m_v0["emissions"].sel(year=slice(2015, 2020)).sel(north_bend_cell).plot(label="v0")
emissions_30m_v1["total_emissions"].sel(north_bend_cell).plot(label="v1")
plt.legend()

# create a one degree roll up to inspect global emissions


In [ ]:
one_degree_raster = coarse_v1.coarsen(lat=40, lon=40).sum().compute()  # .drop("spatial_ref")

# sources annual average


In [ ]:
(
    (coarse_v1["emissions_from_clearing"] + coarse_v1["emissions_from_fire"]).sum(dim="year") / 6
).plot(cmap="fire_light", robust=True)

# sources 1 degree


In [ ]:
(
    (one_degree_raster["emissions_from_clearing"] + one_degree_raster["emissions_from_fire"]).sum(
        dim="year"
    )
    / 6
).plot(cmap="fire_light", robust=True)

# sources 1 degree v0


In [ ]:
one_degree_raster_v0 = coarse_v0.coarsen(lat=40, lon=40).sum().compute()  # .drop("spatial_ref")
(
    (
        one_degree_raster_v0["emissions_from_clearing"]
        + one_degree_raster_v0["emissions_from_fire"]
    ).sum(dim="year")
    / 6
).plot(cmap="fire_light", robust=True)

# sinks at one degree


In [ ]:
(one_degree_raster["sinks"].sum(dim="year") / 6).plot(robust=True, cmap="water_light_r")

# net emissions averaged over 2015-2020


In [ ]:
# net 1 degree
(one_degree_raster.to_array("variable").sum(dim="variable").sum(dim="year") / 6).plot(
    robust=True, cmap="orangeblue_light_r"
)

# or every year separately- the disparities among regions being net sources or sinks gets stronger in 2020


In [ ]:
one_degree_raster.to_array("variable").sum(dim="variable").plot(
    vmin=-1.5e6, vmax=1.5e6, col="year", col_wrap=3, cmap="orangeblue_light_r"
)

In [ ]:
one_degree_raster.sel(year=2020).to_array("variable").sum(dim="variable").plot(
    robust=True, cmap="orangeblue_light_r"
)

# total global emissions timeseries 2015-2020


In [ ]:
coarse_v0["total_emissions"].sel(year=slice(2015, 2020)).sum(dim=["lat", "lon"]).plot(label="v0")
coarse_v1["total_emissions"].sel(year=slice(2015, 2020)).sum(dim=["lat", "lon"]).plot(label="v1")
plt.ylabel("Global emissions \n[tCO2/year]")
plt.xlabel("")
plt.legend()